# 📊 Analyse de Corrélation Dynamique des Cryptomonnaies

## 🎯 Objectif du Projet

Ce notebook présente une **analyse complète des corrélations dynamiques** entre cryptomonnaies majeures pour :
- 📈 **Identifier des groupes** d'actifs avec comportements similaires
- 🔄 **Analyser l'évolution temporelle** des corrélations sur 2 ans
- 🎪 **Détecter les changements de régime** dans les marchés crypto
- 🎯 **Fournir des insights** pour l'optimisation de portefeuille

## 📋 Structure du Notebook
1. Configuration et imports
2. Collecte des données
3. Prétraitement des données
4. Analyse de corrélation dynamique
5. Clustering hiérarchique
6. Visualisation en réseau des corrélations
7. Clustering temporel avec K-Means
8. Détection automatique des changements de régime
9. Clustering spectral optimisé
10. Rapport final et conclusion

In [ ]:
# ================================================================================================
# 📦 1. CONFIGURATION ET IMPORTS
# ================================================================================================

# Suppression des warnings pour une sortie plus propre
import warnings
warnings.filterwarnings('ignore')

# === IMPORTS PRINCIPAUX ===
import datetime
import pandas as pd
import numpy as np

# === VISUALISATION ===
import matplotlib.pyplot as plt
import seaborn as sns

# === DONNÉES FINANCIÈRES ===
import yfinance as yf

# === MACHINE LEARNING & CLUSTERING ===
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.metrics import silhouette_score

# === ANALYSE STATISTIQUE ===
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
from scipy.signal import find_peaks

# === ANALYSE DE RÉSEAUX ===
import networkx as nx
from itertools import combinations

# === CONFIGURATION GRAPHIQUES ===
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams.update({
    'figure.figsize': (14, 8),
    'font.size': 11,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'figure.facecolor': 'white'
})

print("✅ Tous les modules importés avec succès")
print(f"📅 Date d'exécution : {datetime.datetime.now().strftime('%d/%m/%Y %H:%M')}")

# 📦 2. COLLECTE DES DONNÉES

Téléchargement des données historiques sur **2 ans** pour les 9 cryptomonnaies sélectionnées:

| Crypto | Symbole | Description |
|--------|---------|-------------|
| **Ethereum** | ETH-USD | Plateforme smart contracts |
| **Binance Coin** | BNB-USD | Token exchange Binance |
| **Ripple** | XRP-USD | Réseau de paiements |
| **Solana** | SOL-USD | Blockchain haute performance |
| **Cardano** | ADA-USD | Blockchain académique |
| **Polkadot** | DOT-USD | Interopérabilité blockchain |
| **Shiba Inu** | SHIB-USD | Meme coin populaire |
| **Litecoin** | LTC-USD | "L'argent numérique" |
| **Avalanche** | AVAX-USD | Plateforme DeFi |

In [ ]:
def collect_crypto_data(symbols, num_days=730):
    """
    Collecte les données historiques des cryptomonnaies
    
    Args:
        symbols: Liste des symboles de cryptomonnaies
        num_days: Nombre de jours d'historique (défaut: 730 = 2 ans)
    
    Returns:
        DataFrame avec les prix de clôture quotidiens
    """
    print("📈 Collecte des données en cours...")
    
    # Configuration des dates
    start = datetime.date.today() - datetime.timedelta(days=num_days)
    end = datetime.date.today()
    
    print(f"📅 Période : {start} → {end} ({num_days} jours)")
    
    # Téléchargement des données
    data_close = pd.DataFrame()
    success_count = 0
    
    for i, symbol in enumerate(symbols, 1):
        try:
            print(f"   [{i:2d}/{len(symbols)}] {symbol:<10} ... ", end="")
            data = yf.download(symbol, start=start, end=end, interval="1d", progress=False)
            
            if not data.empty:
                data_close[symbol] = data["Close"]
                success_count += 1
                print("✅")
            else:
                print("❌ (Données vides)")
                
        except Exception as e:
            print(f"❌ ({str(e)[:30]}...)")
    
    # Nettoyage des données
    initial_length = len(data_close)
    data_close.dropna(inplace=True)
    final_length = len(data_close)
    
    print(f"\n📊 Résumé :")
    print(f"   • Cryptos collectées : {success_count}/{len(symbols)}")
    print(f"   • Données brutes : {initial_length:,} jours")
    print(f"   • Données nettoyées : {final_length:,} jours")
    print(f"   • Données supprimées : {initial_length - final_length:,} jours")
    
    return data_close

# === CONFIGURATION DES CRYPTOMONNAIES ===
crypto_symbols = [
    "ETH-USD",   # Ethereum
    "BNB-USD",   # Binance Coin
    "XRP-USD",   # Ripple
    "SOL-USD",   # Solana
    "ADA-USD",   # Cardano
    "DOT-USD",   # Polkadot
    "SHIB-USD",  # Shiba Inu
    "LTC-USD",   # Litecoin
    "AVAX-USD"   # Avalanche
]

# === COLLECTE DES DONNÉES ===
data_close = collect_crypto_data(crypto_symbols, num_days=730)

# Aperçu des données
print(f"\n📋 Aperçu des données collectées :")
print(data_close.tail())
print(f"\n📏 Dimensions finales : {data_close.shape}")

# 🧮 3. PRÉTRAITEMENT DES DONNÉES

Calcul des rendements logarithmiques pour normaliser les données et faciliter l'analyse statistique.
Permet également de stabiliser la variance et de réduire l'asymétrie des distributions.

In [ ]:
def calculate_log_returns(price_data):
    """
    Calcule les rendements logarithmiques à partir des prix
    
    Args:
        price_data: DataFrame avec les prix de clôture
    
    Returns:
        DataFrame avec les rendements logarithmiques
    """
    print("🔢 Calcul des rendements logarithmiques...")
    
    # Calcul des rendements logarithmiques
    log_returns = np.log(price_data / price_data.shift(1))
    log_returns.dropna(inplace=True)
    
    print(f"Rendements calculés pour {len(log_returns)} jours")
    
    # Statistiques descriptives
    print("\n📈 Statistiques des rendements:")
    print(f"Rendement moyen : {log_returns.mean().mean():.4f}")
    print(f"Volatilité moyenne : {log_returns.std().mean():.4f}")
    
    return log_returns

# Calcul des rendements
log_returns = calculate_log_returns(data_close)

# Visualisation des rendements
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

# Graphique des prix
data_close.plot(ax=axes[0], title="Évolution des Prix des Cryptomonnaies", 
                legend=True, alpha=0.8)
axes[0].set_ylabel("Prix (USD)")
axes[0].grid(True, alpha=0.3)

# Graphique des rendements
log_returns.plot(ax=axes[1], title="Rendements Logarithmiques", 
                 legend=True, alpha=0.7)
axes[1].set_ylabel("Rendements")
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Prétraitement des données terminé")

# 📊 4. ANALYSE DE CORRÉLATION DYNAMIQUE

Calcul des matrices de corrélation glissantes avec une fenêtre de 30 jours pour capturer l'évolution 
temporelle des relations entre cryptomonnaies.

In [ ]:
def calculate_rolling_correlations(returns_data, window=30):
    """
    Calcule les corrélations glissantes pour analyser l'évolution temporelle
    
    Args:
        returns_data: DataFrame des rendements
        window: Taille de la fenêtre glissante (jours)
    
    Returns:
        Liste des matrices de corrélation, dates correspondantes
    """
    print(f"🔄 Calcul des corrélations glissantes (fenêtre: {window} jours)...")
    
    rolling_corrs = []
    
    # Calcul des corrélations glissantes avec barre de progression
    for i in range(window, len(returns_data) + 1):
        if i % 50 == 0:  # Affichage du progrès
            progress = (i - window + 1) / (len(returns_data) - window + 1) * 100
            print(f"  Progression: {progress:.1f}%")
        
        corr_matrix = returns_data.iloc[i-window:i].corr()
        rolling_corrs.append(corr_matrix)
    
    # Dates correspondantes
    dates = returns_data.index[window-1:]
    
    print(f"✅ {len(rolling_corrs)} matrices de corrélation calculées")
    print(f"Période couverte: {dates[0].date()} à {dates[-1].date()}")
    
    return rolling_corrs, dates

# Calcul des corrélations glissantes
window_size = 30
rolling_correlations, correlation_dates = calculate_rolling_correlations(log_returns, window_size)

# Sauvegarde des données pour usage ultérieur
latest_correlation = rolling_correlations[-1]
latest_correlation.to_csv('last_correlation_matrix.csv')
print(f"\n💾 Dernière matrice de corrélation sauvegardée: last_correlation_matrix.csv")

In [ ]:
def visualize_correlation_evolution(rolling_corrs, dates):
    """
    Visualise l'évolution des corrélations dans le temps
    
    Args:
        rolling_corrs: Liste des matrices de corrélation
        dates: Dates correspondantes
    """
    print("📊 Création des visualisations de corrélation...")
    
    # Configuration de la figure
    fig, axes = plt.subplots(1, 3, figsize=(24, 8))
    
    # Indices pour début, milieu, fin
    indices = [0, len(rolling_corrs)//2, -1]
    labels = ["Début", "Milieu", "Fin"]
    
    for i, (idx, label) in enumerate(zip(indices, labels)):
        correlation_matrix = rolling_corrs[idx]
        date = dates[idx] if idx != -1 else dates[idx]
        
        # Création de la heatmap
        sns.heatmap(correlation_matrix, 
                   annot=True, 
                   cmap="RdBu_r", 
                   center=0,
                   vmin=-1, 
                   vmax=1,
                   ax=axes[i],
                   fmt='.2f',
                   square=True,
                   cbar_kws={'label': 'Corrélation'})
        
        axes[i].set_title(f"Corrélations {label}\n({date.date()})", 
                         fontsize=14, fontweight='bold')
        
        # Rotation des labels pour meilleure lisibilité
        axes[i].set_xticklabels(axes[i].get_xticklabels(), rotation=45, ha='right')
        axes[i].set_yticklabels(axes[i].get_yticklabels(), rotation=0)
    
    plt.suptitle("Évolution des Corrélations entre Cryptomonnaies", 
                 fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    return fig

# Création des visualisations
correlation_figure = visualize_correlation_evolution(rolling_correlations, correlation_dates)

# Analyse comparative début vs fin
print("\n🔍 Analyse comparative des corrélations:")
start_corr = rolling_correlations[0]
end_corr = rolling_correlations[-1]
correlation_change = end_corr - start_corr

print(f"Corrélation moyenne début: {start_corr.mean().mean():.3f}")
print(f"Corrélation moyenne fin: {end_corr.mean().mean():.3f}")
print(f"Changement moyen: {correlation_change.mean().mean():.3f}")

# 🌳 5. CLUSTERING HIÉRARCHIQUE

Application d'un clustering hiérarchique sur la matrice de corrélation la plus récente pour 
identifier des groupes de cryptomonnaies ayant un comportement similaire.

In [ ]:
def perform_hierarchical_clustering(correlation_matrix, method='ward', threshold=1.5):
    """
    Effectue un clustering hiérarchique sur la matrice de corrélation
    
    Args:
        correlation_matrix: Matrice de corrélation
        method: Méthode de linkage
        threshold: Seuil pour former les clusters
    
    Returns:
        Clusters, matrice de linkage, dictionnaire des clusters
    """
    print("🌳 Clustering hiérarchique en cours...")
    
    # Calcul de la matrice de distance adaptée (pour corrélations négatives)
    distance_matrix = np.sqrt(0.5 * (1 - correlation_matrix))
    
    # Conversion en matrice condensée pour linkage
    condensed_distances = squareform(distance_matrix)
    
    # Clustering hiérarchique
    linkage_matrix = linkage(condensed_distances, method=method)
    
    # Visualisation du dendrogramme
    plt.figure(figsize=(14, 8))
    dendrogram(linkage_matrix, 
               labels=correlation_matrix.columns, 
               leaf_rotation=90,
               leaf_font_size=12)
    plt.title(f"Dendrogramme - Clustering Hiérarchique ({method.title()})", 
              fontsize=14, fontweight='bold')
    plt.xlabel("Cryptomonnaies", fontsize=12)
    plt.ylabel("Distance", fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Attribution des clusters
    clusters = fcluster(linkage_matrix, t=threshold, criterion='distance')
    
    # Affichage des résultats
    print(f"\n📊 Résultats du clustering (seuil: {threshold}):")
    cluster_dict = {}
    for crypto, cluster in zip(correlation_matrix.columns, clusters):
        if cluster not in cluster_dict:
            cluster_dict[cluster] = []
        cluster_dict[cluster].append(crypto)
    
    for cluster_id, cryptos in cluster_dict.items():
        print(f"  Cluster {cluster_id}: {', '.join(cryptos)}")
    
    return clusters, linkage_matrix, cluster_dict

# Application du clustering hiérarchique
hierarchical_clusters, linkage_matrix, cluster_groups = perform_hierarchical_clustering(
    latest_correlation, method='ward', threshold=1.5
)

# 🕸️ 6. VISUALISATION EN RÉSEAU DES CORRÉLATIONS

Création d'un graphe pour visualiser les corrélations fortes entre cryptomonnaies (seuil > 0.6).
Cette représentation permet de mieux comprendre les relations entre les actifs.

In [ ]:
def create_correlation_network(correlation_matrix, threshold=0.6, layout_seed=42):
    """
    Crée un graphe des corrélations fortes entre cryptomonnaies
    
    Args:
        correlation_matrix: Matrice de corrélation
        threshold: Seuil de corrélation pour créer une arête
        layout_seed: Graine pour la disposition du graphe
    
    Returns:
        Graphe NetworkX
    """
    print(f"🕸️ Création du graphe de corrélations (seuil: {threshold})...")
    
    # Création du graphe
    G = nx.Graph()
    
    # Ajout des arêtes pour corrélations > seuil
    edges_added = 0
    for i, j in combinations(correlation_matrix.columns, 2):
        correlation_value = correlation_matrix.loc[i, j]
        if correlation_value > threshold:
            G.add_edge(i, j, weight=correlation_value)
            edges_added += 1
    
    print(f"  {edges_added} connexions fortes détectées")
    
    if edges_added == 0:
        print(f"  ⚠️ Aucune corrélation > {threshold} trouvée. Réduction du seuil...")
        threshold = 0.4
        for i, j in combinations(correlation_matrix.columns, 2):
            correlation_value = correlation_matrix.loc[i, j]
            if correlation_value > threshold:
                G.add_edge(i, j, weight=correlation_value)
                edges_added += 1
        print(f"  {edges_added} connexions détectées avec seuil réduit à {threshold}")
    
    # Calcul de la disposition
    pos = nx.spring_layout(G, seed=layout_seed, k=3, iterations=50)
    
    # Visualisation
    plt.figure(figsize=(12, 10))
    
    # Calcul des poids des arêtes pour la visualisation
    edge_weights = [G[u][v]['weight'] * 8 for u, v in G.edges()]
    edge_colors = [G[u][v]['weight'] for u, v in G.edges()]
    
    # Dessin du graphe
    nx.draw_networkx_nodes(G, pos, 
                          node_color='lightblue', 
                          node_size=2000, 
                          alpha=0.8)
    
    nx.draw_networkx_edges(G, pos, 
                          width=edge_weights, 
                          edge_color=edge_colors,
                          edge_cmap=plt.cm.Reds,
                          alpha=0.7)
    
    nx.draw_networkx_labels(G, pos, 
                           font_size=12, 
                           font_weight='bold')
    
    # Ajout des poids sur les arêtes
    edge_labels = {(u, v): f"{G[u][v]['weight']:.2f}" for u, v in G.edges()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=10)
    
    plt.title(f"Réseau des Corrélations Fortes (> {threshold})", 
              fontsize=14, fontweight='bold')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Statistiques du réseau
    print(f"\n📈 Statistiques du réseau:")
    print(f"  Nœuds: {G.number_of_nodes()}")
    print(f"  Arêtes: {G.number_of_edges()}")
    print(f"  Densité: {nx.density(G):.3f}")
    
    return G

# Création du graphe de corrélations
correlation_network = create_correlation_network(latest_correlation, threshold=0.6)

# ⏰ 7. CLUSTERING TEMPOREL AVEC K-MEANS

Application de K-Means sur les matrices de corrélation aplaties pour identifier des régimes 
temporels distincts dans les relations entre cryptomonnaies.

In [ ]:
def perform_temporal_kmeans_clustering(rolling_correlations, n_clusters=3):
    """
    Effectue un clustering K-Means temporel sur les matrices de corrélation
    
    Args:
        rolling_correlations: Liste des matrices de corrélation
        n_clusters: Nombre de clusters
    
    Returns:
        Labels des clusters, modèle K-Means, données aplaties
    """
    print(f"⏰ Clustering temporel K-Means ({n_clusters} clusters)...")
    
    # Fonction pour aplatir la matrice de corrélation (sans diagonale)
    def flatten_correlation_matrix(correlation_matrix):
        mask = ~np.eye(correlation_matrix.shape[0], dtype=bool)
        return correlation_matrix.values[mask]
    
    # Aplatissement de toutes les matrices
    flattened_data = np.array([
        flatten_correlation_matrix(corr_matrix) 
        for corr_matrix in rolling_correlations
    ])
    
    print(f"  Données préparées: {flattened_data.shape}")
    
    # Application du K-Means
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans_model.fit_predict(flattened_data)
    
    # Calcul des statistiques
    unique_labels, counts = np.unique(cluster_labels, return_counts=True)
    print(f"  Répartition des clusters:")
    for label, count in zip(unique_labels, counts):
        percentage = count / len(cluster_labels) * 100
        print(f"    Cluster {label+1}: {count} jours ({percentage:.1f}%)")
    
    return cluster_labels, kmeans_model, flattened_data

def visualize_temporal_clusters(dates, cluster_labels, n_clusters=3):
    """
    Visualise l'évolution temporelle des clusters
    
    Args:
        dates: Dates correspondantes
        cluster_labels: Labels des clusters
        n_clusters: Nombre de clusters
    """
    plt.figure(figsize=(15, 6))
    
    # Couleurs pour chaque cluster
    colors = plt.cm.Set1(np.linspace(0, 1, n_clusters))
    
    # Graphique principal
    for i in range(n_clusters):
        mask = cluster_labels == i
        plt.scatter(dates[mask], cluster_labels[mask] + 1, 
                   c=[colors[i]], label=f'Régime {i+1}', 
                   alpha=0.7, s=30)
    
    plt.ylabel("Régime de Corrélation", fontsize=12)
    plt.xlabel("Date", fontsize=12)
    plt.title("Évolution des Régimes de Corrélation (K-Means Temporel)", 
              fontsize=14, fontweight='bold')
    plt.yticks(range(1, n_clusters+1), [f'Régime {i+1}' for i in range(n_clusters)])
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Application du clustering temporel
temporal_labels, kmeans_model, correlation_data = perform_temporal_kmeans_clustering(
    rolling_correlations, n_clusters=3
)

# Visualisation
visualize_temporal_clusters(correlation_dates, temporal_labels, n_clusters=3)

# 🔍 8. DÉTECTION AUTOMATIQUE DES CHANGEMENTS DE RÉGIME

Utilisation de l'analyse de variance et de la détection de pics pour identifier 
automatiquement les périodes de changement structurel dans les corrélations.

In [ ]:
def detect_regime_changes(rolling_correlations, dates):
    """
    Détecte automatiquement les changements de régime basés sur la variance des corrélations
    
    Args:
        rolling_correlations: Liste des matrices de corrélation
        dates: Dates correspondantes
    
    Returns:
        Indices des changements de régime, variance temporelle
    """
    print("🔍 Détection automatique des changements de régime...")
    
    # Calcul de la variance moyenne des corrélations dans le temps
    variance_timeline = []
    
    for correlation_matrix in rolling_correlations:
        # Extraction de la partie supérieure de la matrice (sans diagonale)
        upper_triangle = correlation_matrix.where(
            np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
        )
        variance = np.nanvar(upper_triangle.values.flatten())
        variance_timeline.append(variance)
    
    variance_timeline = np.array(variance_timeline)
    
    # Détection des pics de variance (changements de régime potentiels)
    threshold_height = np.mean(variance_timeline) + 1.5 * np.std(variance_timeline)
    regime_change_indices, properties = find_peaks(
        variance_timeline, 
        height=threshold_height,
        distance=10  # Distance minimale entre pics
    )
    
    # Visualisation
    plt.figure(figsize=(16, 8))
    
    # Graphique principal
    plt.plot(dates, variance_timeline, 
             label='Variance des Corrélations', 
             linewidth=2, color='blue', alpha=0.7)
    
    # Ligne de seuil
    plt.axhline(y=threshold_height, color='red', linestyle='--', 
                label=f'Seuil de détection ({threshold_height:.4f})', alpha=0.7)
    
    # Points de changement de régime
    if len(regime_change_indices) > 0:
        plt.scatter(dates[regime_change_indices], 
                   variance_timeline[regime_change_indices],
                   color='red', s=100, marker='v', 
                   label='Changements de Régime Détectés', 
                   zorder=5)
        
        # Annotations
        for idx in regime_change_indices:
            plt.annotate(f'{dates[idx].strftime("%Y-%m-%d")}',
                        (dates[idx], variance_timeline[idx]),
                        xytext=(10, 10), textcoords='offset points',
                        bbox=dict(boxstyle='round,pad=0.3', fc='yellow', alpha=0.7),
                        fontsize=10)
    
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Variance des Corrélations', fontsize=12)
    plt.title('Détection Automatique des Changements de Régime', 
              fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Résultats
    print(f"\n📊 Résultats de la détection:")
    print(f"  Changements de régime détectés: {len(regime_change_indices)}")
    print(f"  Variance moyenne: {np.mean(variance_timeline):.4f}")
    print(f"  Variance maximale: {np.max(variance_timeline):.4f}")
    
    if len(regime_change_indices) > 0:
        print(f"\n📅 Dates des changements détectés:")
        for idx in regime_change_indices:
            date_str = dates[idx].strftime("%Y-%m-%d")
            variance_val = variance_timeline[idx]
            print(f"    {date_str} (variance: {variance_val:.4f})")
    else:
        print(f"  ℹ️ Aucun changement de régime significatif détecté")
    
    return regime_change_indices, variance_timeline

# Détection des changements de régime
regime_changes, variance_data = detect_regime_changes(rolling_correlations, correlation_dates)

# 🌟 9. CLUSTERING SPECTRAL OPTIMISÉ

Application du clustering spectral avec optimisation du nombre de clusters par analyse du score silhouette.
Cette méthode est particulièrement adaptée aux données non-convexes et aux structures complexes.

In [ ]:
def perform_spectral_clustering(correlation_matrix, max_clusters=6):
    """
    Effectue un clustering spectral optimisé sur la matrice de corrélation
    
    Args:
        correlation_matrix: Matrice de corrélation
        max_clusters: Nombre maximum de clusters à tester
    
    Returns:
        Labels des clusters, nombre optimal de clusters, scores silhouette
    """
    print("🌟 Clustering spectral avec optimisation...")
    
    # Conversion de la matrice de corrélation en matrice de similarité
    similarity_matrix = (correlation_matrix + 1) / 2  # Normalisation entre 0 et 1
    
    # Test de différents nombres de clusters
    n_clusters_range = range(2, min(max_clusters+1, len(correlation_matrix.columns)))
    silhouette_scores = []
    all_labels = []
    
    for n_clusters in n_clusters_range:
        spectral = SpectralClustering(
            n_clusters=n_clusters, 
            affinity='precomputed', 
            random_state=42
        )
        labels = spectral.fit_predict(similarity_matrix)
        score = silhouette_score(similarity_matrix, labels, metric='precomputed')
        silhouette_scores.append(score)
        all_labels.append(labels)
        
        print(f"  {n_clusters} clusters → Score silhouette: {score:.3f}")
    
    # Visualisation des scores
    plt.figure(figsize=(10, 6))
    plt.plot(n_clusters_range, silhouette_scores, marker='o', linewidth=2, markersize=8)
    plt.grid(True, alpha=0.3)
    plt.xlabel('Nombre de clusters', fontsize=12)
    plt.ylabel('Score silhouette', fontsize=12)
    plt.title('Optimisation du nombre de clusters (Spectral Clustering)', 
              fontsize=14, fontweight='bold')
    plt.xticks(list(n_clusters_range))
    plt.tight_layout()
    plt.show()
    
    # Sélection du meilleur nombre de clusters
    best_idx = np.argmax(silhouette_scores)
    optimal_n_clusters = list(n_clusters_range)[best_idx]
    optimal_labels = all_labels[best_idx]
    
    # Affichage des résultats
    print(f"\n🎯 Nombre optimal de clusters: {optimal_n_clusters} (score: {silhouette_scores[best_idx]:.3f})")
    
    # Organisation des résultats par groupe
    spectral_groups = {}
    for i, crypto in enumerate(correlation_matrix.columns):
        cluster_id = optimal_labels[i]
        if cluster_id not in spectral_groups:
            spectral_groups[cluster_id] = []
        spectral_groups[cluster_id].append(crypto)
    
    for cluster_id, cryptos in spectral_groups.items():
        print(f"  Groupe {cluster_id + 1}: {', '.join(cryptos)}")
    
    return optimal_labels, optimal_n_clusters, silhouette_scores, spectral_groups

# Application du clustering spectral optimisé
spectral_labels, optimal_n_clusters, silhouette_scores, spectral_groups = perform_spectral_clustering(
    latest_correlation, max_clusters=6
)

# 📋 10. RAPPORT FINAL ET CONCLUSION

Synthèse des analyses effectuées et des résultats obtenus.

In [ ]:
def generate_final_report():
    """
    Génère un rapport final avec tous les résultats de l'analyse
    """
    print("\n" + "="*65)
    print("📋 RAPPORT FINAL - ANALYSE DE CORRÉLATION DES CRYPTOMONNAIES")
    print("="*65)
    
    # 1. Résumé des données
    print(f"\n📊 DONNÉES ANALYSÉES:")
    print(f"  • Cryptomonnaies: {len(crypto_symbols)}")
    print(f"  • Période d'analyse: {correlation_dates[0].date()} à {correlation_dates[-1].date()}")
    print(f"  • Nombre de jours: {len(data_close)}")
    print(f"  • Matrices de corrélation calculées: {len(rolling_correlations)}")
    
    # 2. Résultats du clustering hiérarchique
    print(f"\n🌳 CLUSTERING HIÉRARCHIQUE:")
    for cluster_id, cryptos in cluster_groups.items():
        print(f"    Groupe {cluster_id}: {', '.join(cryptos)}")
    
    # 3. Résultats du clustering spectral
    print(f"\n🌟 CLUSTERING SPECTRAL OPTIMAL ({optimal_n_clusters} clusters):")
    for cluster_id, cryptos in spectral_groups.items():
        print(f"    Groupe {cluster_id + 1}: {', '.join(cryptos)}")
    
    # 4. Changements de régime
    print(f"\n🔍 DÉTECTION DE RÉGIMES:")
    print(f"  • Changements de régime détectés: {len(regime_changes)}")
    if len(regime_changes) > 0:
        for idx in regime_changes:
            print(f"    - {correlation_dates[idx].strftime('%Y-%m-%d')}")
    
    # 5. Statistiques temporelles
    temporal_counts = np.bincount(temporal_labels)
    print(f"\n⏰ RÉGIMES TEMPORELS (K-Means):")
    for i, count in enumerate(temporal_counts):
        percentage = count / len(temporal_labels) * 100
        print(f"    Régime {i+1}: {count} jours ({percentage:.1f}%)")
    
    # 6. Métriques de qualité
    correlation_mean = latest_correlation.mean().mean()
    correlation_std = np.std([corr.mean().mean() for corr in rolling_correlations])
    
    print(f"\n📊 MÉTRIQUES DE QUALITÉ:")
    print(f"  • Corrélation moyenne actuelle: {correlation_mean:.3f}")
    print(f"  • Volatilité des corrélations: {correlation_std:.3f}")
    print(f"  • Score silhouette spectral: {silhouette_scores[optimal_n_clusters-2]:.3f}")
    
    # 7. Recommandations
    print(f"\n💡 RECOMMANDATIONS:")
    print(f"  • Diversification: Investir dans différents clusters pour réduire le risque")
    print(f"  • Surveillance: Monitorer les changements de régime pour ajuster la stratégie")
    print(f"  • Gestion des risques: Vigilance accrue en période de forte corrélation")
    
    print(f"\n✅ ANALYSE TERMINÉE AVEC SUCCÈS")
    print("="*65)

# Sauvegarde des résultats
def save_all_results():
    """
    Sauvegarde tous les résultats dans des fichiers CSV
    """
    print("\n💾 Sauvegarde des résultats finaux...")
    
    try:
        # 1. Rendements logarithmiques
        log_returns.to_csv('log_returns_final.csv')
        print("  ✅ log_returns_final.csv")
        
        # 2. Résultats du clustering
        clustering_results = pd.DataFrame({
            'cryptocurrency': latest_correlation.columns,
            'hierarchical_cluster': hierarchical_clusters,
            'spectral_cluster': spectral_labels + 1,  # +1 pour commencer à 1
            'temporal_cluster_final': temporal_labels[-1] + 1
        })
        clustering_results.to_csv('clustering_results_final.csv', index=False)
        print("  ✅ clustering_results_final.csv")
        
        # 3. Détection de régimes
        regime_data = pd.DataFrame({
            'date': correlation_dates,
            'variance': variance_data,
            'temporal_cluster': temporal_labels + 1
        })
        regime_data.to_csv('regime_detection_results.csv', index=False)
        print("  ✅ regime_detection_results.csv")
        
        print(f"\n🎯 Tous les fichiers sauvegardés avec succès!")
        
    except Exception as e:
        print(f"  ❌ Erreur lors de la sauvegarde: {e}")

# Génération du rapport final
generate_final_report()

# Sauvegarde des résultats
save_all_results()

# 💡 CONCLUSION ET RECOMMANDATIONS

## 🎯 Principaux Résultats

L'analyse de corrélation dynamique des cryptomonnaies a révélé plusieurs insights importants :

1. **Groupes de Cryptomonnaies** : Le clustering a identifié des groupes distincts avec des comportements corrélés
2. **Évolution Temporelle** : Les corrélations ne sont pas statiques et évoluent significativement 
3. **Changements de Régime** : Des périodes distinctes avec des structures de corrélation différentes
4. **Diversification** : Possibilités d'optimisation de portefeuille basées sur les groupes identifiés

## 💼 Recommandations pour les Investisseurs

- **Diversification** : Répartir les investissements entre différents clusters
- **Surveillance** : Monitorer les changements de régime pour ajuster la stratégie
- **Gestion des Risques** : Être vigilant pendant les périodes de forte corrélation
- **Opportunités** : Exploiter les décorrélations temporaires pour des stratégies de trading

## 🔮 Améliorations Futures

- Intégration de données macroéconomiques
- Modèles prédictifs pour anticiper les changements
- Analyse en temps réel avec APIs
- Extension à d'autres classes d'actifs

---

**Projet réalisé dans le cadre de l'analyse de données financières - Data Mining**

*Date de finalisation : Mai 2025*